In [1]:
import numpy as np
import torch
import torch.nn as nn

In [2]:
kps = np.ones((300, 17, 3)).astype(np.float32)
kps[:, 13:, 2] = 0.1

In [3]:
kps[0]

array([[1. , 1. , 1. ],
       [1. , 1. , 1. ],
       [1. , 1. , 1. ],
       [1. , 1. , 1. ],
       [1. , 1. , 1. ],
       [1. , 1. , 1. ],
       [1. , 1. , 1. ],
       [1. , 1. , 1. ],
       [1. , 1. , 1. ],
       [1. , 1. , 1. ],
       [1. , 1. , 1. ],
       [1. , 1. , 1. ],
       [1. , 1. , 1. ],
       [1. , 1. , 0.1],
       [1. , 1. , 0.1],
       [1. , 1. , 0.1],
       [1. , 1. , 0.1]], dtype=float32)

In [4]:
def create_mask(kps):
    mask = np.where(
        kps[:, :, 2] < 0.2, -1e10, 0.0
    )  # -inf to nan in softmax of attention module
    seq_len, points = mask.shape
    mask = np.repeat(mask, 2, axis=1).reshape(seq_len, points, 2)
    return mask

In [5]:
mask = create_mask(kps)

In [6]:
mask[-1]

array([[ 0.e+00,  0.e+00],
       [ 0.e+00,  0.e+00],
       [ 0.e+00,  0.e+00],
       [ 0.e+00,  0.e+00],
       [ 0.e+00,  0.e+00],
       [ 0.e+00,  0.e+00],
       [ 0.e+00,  0.e+00],
       [ 0.e+00,  0.e+00],
       [ 0.e+00,  0.e+00],
       [ 0.e+00,  0.e+00],
       [ 0.e+00,  0.e+00],
       [ 0.e+00,  0.e+00],
       [ 0.e+00,  0.e+00],
       [-1.e+10, -1.e+10],
       [-1.e+10, -1.e+10],
       [-1.e+10, -1.e+10],
       [-1.e+10, -1.e+10]])

In [75]:
mask = torch.tensor([[[0, 1]]])
mask.shape

torch.Size([1, 1, 2])

In [76]:
data = torch.tensor([[[0.5, 0.5, 0.5], [0.5, 0.5, 0.5]]]).float()

In [77]:
s = torch.matmul(data, data.transpose(1, 2))
s

tensor([[[0.7500, 0.7500],
         [0.7500, 0.7500]]])

In [80]:
s = s.masked_fill(mask == 0, -1e10)
s

tensor([[[-1.0000e+10,  7.5000e-01],
         [-1.0000e+10,  7.5000e-01]]])

In [84]:
softmax = nn.Softmax(dim=-1)
w = softmax(s)
w

tensor([[[0., 1.],
         [0., 1.]]])

In [85]:
ret = torch.matmul(w, data)
ret

tensor([[[0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000]]])

In [63]:
ret, w = nn.MultiheadAttention(3, 1, batch_first=True)(data, data, data)
ret

tensor([[[-0.1158,  0.0899, -0.0555],
         [-0.1163,  0.0904, -0.0559]]], grad_fn=<TransposeBackward0>)

In [64]:
w

tensor([[[0.5100, 0.4900],
         [0.5071, 0.4929]]], grad_fn=<DivBackward0>)